# Project description

**Status:** Done. Need to add more comments & description. 

## READ before running the notebook
If you are using jupyter notebook and you have variable inspector ticked in your nbextensions config (check in edit drop down menu, last option), I highly recommand to untick it before running this notebook. As some variables of this notebook are pd.DataFrame of dozens of millions of rows, it slows down the whole notebook. 


## Description:
This notebook has for purpose to download [IMDB's public datasets](https://www.imdb.com/interfaces/) which contain 7 `.tsv` files. This table is the the original files not the new one. Need to be updated for after [4 Cleaning each files](#Cleaning-each-files): 


File name / feature name | Feature description
--- | --- 
**akas**  |  Contains the following information for titles:
titleId (string)  |  a tconst, an alphanumeric unique identifier of the title
ordering (integer)  |  a number to uniquely identify rows for a given titleId
title (string)  |  the localized title
region (string)  |  the region for this version of the title
language (string)  |  the language of the title
types (array)  |  Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
attributes (array)  |  Additional terms to describe this alternative title, not enumerated
isOriginalTitle (boolean)  |  0: not original title; 1: original title
|
|
**basics**  |  Contains the following information for titles:
tconst (string)  |  alphanumeric unique identifier of the title
titleType (string)  |  the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
primaryTitle (string)  |  the more popular title / the title used by the filmmakers on promotional materials at the point of release
originalTitle (string)  |  original title, in the original language
isAdult (boolean)  |  0: non-adult title; 1: adult title
startYear (YYYY)  |  represents the release year of a title. In the case of TV Series, it is the series start year
endYear (YYYY)  |  TV Series end year. for all other title types
runtimeMinutes  |  primary runtime of the title, in minutes
genres (string array)  |  includes up to three genres associated with the title
|
|
**crew**  |  Contains the director and writer information for all the titles in IMDb. Fields include:
tconst (string)  |  alphanumeric unique identifier of the title
directors (array of nconsts)  |  director(s) of the given title
writers (array of nconsts)  |  writer(s) of the given title
|
|
**episode**  |  Contains the tv episode information. Fields include:
tconst (string)  |  alphanumeric identifier of episode
parentTconst (string)  |  alphanumeric identifier of the parent TV Series
seasonNumber (integer)  |  season number the episode belongs to
episodeNumber (integer)  |  episode number of the tconst in the TV series
|
|
**principals**  |  Contains the principal cast/crew for titles
tconst (string)  |  alphanumeric unique identifier of the title
ordering (integer)  |  a number to uniquely identify rows for a given titleId
nconst (string)  |  alphanumeric unique identifier of the name/person
category (string)  |  the category of job that person was in
job (string)  |  the specific job title if applicable, else 
characters (string)  |  the name of the character played if applicable, else 
|
|
**ratings**  |  Contains the IMDb rating and votes information for titles
tconst (string)  |  alphanumeric unique identifier of the title
averageRating  |  weighted average of all the individual user ratings
numVotes  |  number of votes the title has received
|
|
**name**  |  Contains the following information for names:
nconst (string)  |  alphanumeric unique identifier of the name/person
primaryName (string)  |  name by which the person is most often credited
birthYear  |  in YYYY format
deathYear  |  in YYYY format if applicable, else 
primaryProfession (array of strings)  |  the top-3 professions of the person
knownForTitles (array of tconsts)  |  titles the person is known for






**Run time on Macbook Pro 2017, i5 3.1 GHz 2 cores, 8GB of ram:** ~40mn

**Macbook Air M1 🥺BG of ram:** 🥺

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

# Get Imdb files links


## Import libraries 

* Requests to get the page 
* Beautiful Soup to get the content of that page
* Os and Shutil for file management
* Patoolib to decompress .gz archives
* Time To know how long it takes to excecute the script 
* Termcolor for unnecessary beautiful colored print statements 
* Re for string manipulation
* Caffeine for display always on

In [1]:
from bs4 import BeautifulSoup as bs
import requests

import os
from os import listdir
from os.path import isfile, join
import shutil
import patoolib as patoo
import re

import pandas as pd
import numpy as np

import time

from termcolor import colored

import caffeine
def on(): # Shortcut def to let the display on while runing long code. 
    caffeine.on(display=False)

## Scrape files links

You know when you click on a link and it doesn't open a new page but download a file? That is what's happening my friend, we're getting those links!

In [55]:
start = time.time()

# Link where to find the datasets 
url = "https://datasets.imdbws.com/"
links = []

# Get the web page
page = requests.get(url)

# Get the page's html 
soup = bs(page.content, "html.parser")

# Get all text as href tag
for href in soup.find_all("a"):
    links.append(href["href"])
    
links

['http://www.imdb.com/interfaces/',
 'https://datasets.imdbws.com/name.basics.tsv.gz',
 'https://datasets.imdbws.com/title.akas.tsv.gz',
 'https://datasets.imdbws.com/title.basics.tsv.gz',
 'https://datasets.imdbws.com/title.crew.tsv.gz',
 'https://datasets.imdbws.com/title.episode.tsv.gz',
 'https://datasets.imdbws.com/title.principals.tsv.gz',
 'https://datasets.imdbws.com/title.ratings.tsv.gz']

## Store them

In [56]:
down_links = [x for x in links if "tsv" in x]
metadata_link = list(set(links) - (set(down_links)))[0]

metadata_link

'http://www.imdb.com/interfaces/'

In [57]:
down_links

['https://datasets.imdbws.com/name.basics.tsv.gz',
 'https://datasets.imdbws.com/title.akas.tsv.gz',
 'https://datasets.imdbws.com/title.basics.tsv.gz',
 'https://datasets.imdbws.com/title.crew.tsv.gz',
 'https://datasets.imdbws.com/title.episode.tsv.gz',
 'https://datasets.imdbws.com/title.principals.tsv.gz',
 'https://datasets.imdbws.com/title.ratings.tsv.gz']

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />


# Download files

## Create new directory to store them

In [58]:
directory = "data/TSVs"
if not os.path.exists(directory):
    os.makedirs(directory)

## Download the files

In [59]:
print('Download Starting...')
start = time.time() # Starting timer

for url in down_links:
    start2 = time.time()

    # Download files
    r = requests.get(url)
    filename = url.split('/')[-1].replace(".", "_", 1)
    print(filename)

    with open(filename,'wb') as output_file:
        output_file.write(r.content)

        
    # Decompress file
    patoo.extract_archive(filename, outdir="")

    # Delete compressed file
    os.remove(filename)

    # Move file to the right folder
    filename = filename.replace(".gz", "")
    shutil.move(filename, directory)


    index = down_links.index(url) + 1
    print(colored(f"{filename} done, {len(down_links) - index} more - {round(time.time() - start2)}s", "red"))

print(colored(f"Download Completed!!! It took {round(time.time() - start)} seconds", "red", attrs=['bold']))

Download Starting...
name_basics.tsv.gz
patool: Extracting name_basics.tsv.gz ...
patool: running '/usr/bin/gzip' -c -d -- 'name_basics.tsv.gz' > 'name_basics.tsv'
patool:     with shell='True'
patool: ... name_basics.tsv.gz extracted to `'.
name_basics.tsv done, 6 more - 9s
title_akas.tsv.gz
patool: Extracting title_akas.tsv.gz ...
patool: running '/usr/bin/gzip' -c -d -- 'title_akas.tsv.gz' > 'title_akas.tsv'
patool:     with shell='True'
patool: ... title_akas.tsv.gz extracted to `'.
title_akas.tsv done, 5 more - 11s
title_basics.tsv.gz
patool: Extracting title_basics.tsv.gz ...
patool: running '/usr/bin/gzip' -c -d -- 'title_basics.tsv.gz' > 'title_basics.tsv'
patool:     with shell='True'
patool: ... title_basics.tsv.gz extracted to `'.
title_basics.tsv done, 4 more - 10s
title_crew.tsv.gz
patool: Extracting title_crew.tsv.gz ...
patool: running '/usr/bin/gzip' -c -d -- 'title_crew.tsv.gz' > 'title_crew.tsv'
patool:     with shell='True'
patool: ... title_crew.tsv.gz extracted to 

## Checking files

Here I just check that all the seven files that were links in "links" are in the directory in the right format

In [60]:
onlyfiles = [f for f in listdir(directory) if isfile(join(directory, f)) and ".tsv" in f]

for file in onlyfiles:
    print(file)

title_basics.tsv
title_ratings.tsv
title_crew.tsv
name_basics.tsv
title_akas.tsv
title_episode.tsv
title_principals.tsv


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />



# Cleaning each files

## Basics

### Import file

In [61]:
df_basics = pd.read_csv("data/TSVs/title_basics.tsv", delimiter="\t")
df_basics.shape

(7948647, 9)

In [62]:
df_basics.columns = ["t_id", "type", "primary_title", "original_title", "for_adult", 
                     "start_year", "end_year", "runtime_mn", "genres"]

### Replacing missing values & title_id

* Missing avlues are represented by `\N` and I prefer to have missing values instead as `\n`
* Changing title_id from `tt0000001` to `1`, `tt000389` to `389`,  and so on. (outdated)

In [63]:
def clean_df(df):
    df.replace(r"\N", np.nan, inplace=True)
        
    return df

df_basics = clean_df(df_basics)
df_basics.head()

,t_id,type,primary_title,original_title,for_adult,start_year,end_year,runtime_mn,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [64]:
# for genre in ['Reality-TV', 'Documentary', 'Talk-Show', 'Game-Show', 'Animation,Comedy,Family']:
#     indexes = df_basics[df_basics["runtime_mn"] == genre].index
    
#     for index in indexes: 
#         df_basics.loc[index, "genres"] = genre

### Changing data types


In [65]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7948647 entries, 0 to 7948646
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   t_id            object
 1   type            object
 2   primary_title   object
 3   original_title  object
 4   for_adult       object
 5   start_year      object
 6   end_year        object
 7   runtime_mn      object
 8   genres          object
dtypes: object(9)
memory usage: 545.8+ MB


#### Changing Year from `float` to `datetime`

In [66]:
df_basics["start_year"] = df_basics["start_year"].astype("datetime64")
df_basics["end_year"] = df_basics["end_year"].astype("datetime64")

#### Changing others from `float` to `int`

In the function, there are two ways of doing it. The first way with `list`seems to be way faster than imputing at each index of the `Series` using `.loc`.

In [67]:
def series_to_int(df, *args):
    for col in args:
        liste = df[col].to_list() # Used lists as pd.Series were too slow
        
        new = []
        for x in range(len(liste)):
            val = liste[x]
            try:
                new.append(int(val))
            except: 
                new.append(np.nan)
                
            if x % 100000 == 0:
                print(f"{round(x / len(liste) * 100, 2)}%", " " * 100, end="\r")
        df[col] = new
    

        df[col] = df[col].astype("Int64")

    return df

df_basics = series_to_int(df_basics, "for_adult", "runtime_mn")

In [68]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7948647 entries, 0 to 7948646
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   t_id            object        
 1   type            object        
 2   primary_title   object        
 3   original_title  object        
 4   for_adult       Int64         
 5   start_year      datetime64[ns]
 6   end_year        datetime64[ns]
 7   runtime_mn      Int64         
 8   genres          object        
dtypes: Int64(2), datetime64[ns](2), object(5)
memory usage: 561.0+ MB


### Getting all genres 

So we can have a new DataFrame that will have each genre for each movie!

In [69]:
df_genres = df_basics[["t_id", "genres"]].set_index("t_id")["genres"].str.split(",").explode().to_frame().reset_index()
df_genres.shape

(12619654, 2)

In [70]:
df_genres.head()

,t_id,genres
0,tt0000001,Documentary
1,tt0000001,Short
2,tt0000002,Animation
3,tt0000002,Short
4,tt0000003,Animation


### Export both tables

Keeping the genre column with list of genre doesn't lead to anything so I'm going to create two new tables.
* genre that will contain each and individual genre
* title_id that will contain each title_id (one observation for each genre of each title)


In [71]:
df_genres.to_csv("data/genres.csv", index=False)
print("Genre table exported!")

# We don't need genre column in basics as we have a new df for it 
df_basics.drop("genres", axis=1).to_csv("data/basics.csv", index=False) # Exported as csv for convenience 
print("Basics table exported!")

# We also don't need that tsv file anymore
# os.remove(directory + "/title_basics.tsv")

Genre table exported!
Basics table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Principals
### Import file

In [145]:
df_principals = pd.read_csv("data/TSVs/title_principals.tsv", delimiter="\t")
df_principals.shape

KeyboardInterrupt: 

In [73]:
df_principals.columns = ['t_id', 'ordering', 'n_id', 'job_category', 'job_title', 'character_played']

### Replace missing values & id of id columns

In [74]:
df_principals = clean_df(df_principals)
df_principals.head()

,t_id,ordering,n_id,job_category,job_title,character_played
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


**No need to change the data types.**

In [75]:
df_principals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44994526 entries, 0 to 44994525
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   t_id              object
 1   ordering          int64 
 2   n_id              object
 3   job_category      object
 4   job_title         object
 5   character_played  object
dtypes: int64(1), object(5)
memory usage: 2.0+ GB


### Export title_principals 

Now that we have correct missing values and ids, let's export it!

In [76]:
# We don't need ordering column
df_principals.drop("ordering", axis=1).to_csv("data/principals.csv", index=False)
print("Principals table exported!")

# os.remove(directory + "/title_principals.tsv")

Principals table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Ratings

Finally a light file... The lightest actually!

### Import file

In [77]:
df_ratings = pd.read_csv("data/TSVs/title_ratings.tsv", delimiter="\t")
df_ratings.shape

(1157108, 3)

In [78]:
df_ratings.columns = ['t_id', 'rating', 'votes']

### Replacing missing values & id of id_columns


In [79]:
df_ratings = clean_df(df_ratings)
df_ratings.head()

,t_id,rating,votes
0,tt0000001,5.7,1707
1,tt0000002,6.1,210
2,tt0000003,6.5,1465
3,tt0000004,6.2,123
4,tt0000005,6.2,2266


**No need to change the data types.**

In [80]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157108 entries, 0 to 1157107
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   t_id    1157108 non-null  object 
 1   rating  1157108 non-null  float64
 2   votes   1157108 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 26.5+ MB


### Export Ratings 

Now that we have correct missing values and ids, let's export it!

In [81]:
df_ratings.to_csv("data/ratings.csv", index=False)
print("Ratings table exported!")

# We also don't need that tsv file anymore
# os.remove(directory + "/title_ratings.tsv")

Ratings table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Episode

### Import file

In [82]:
df_episode = pd.read_csv("data/TSVs/title_episode.tsv", delimiter="\t")
df_episode.shape

(5799623, 4)

`parent_id` is the name of the TV-show where `t_id` is the title of the episode. For movies, `t_id` is the title of the movie.


In [83]:
df_episode.columns = ['t_id', 'parent_id', 'season', 'episode']

### Replacing missing values & id of id_columns


In [84]:
df_episode = clean_df(df_episode)
df_episode.head()

,t_id,parent_id,season,episode
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


In [85]:
df_episode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5799623 entries, 0 to 5799622
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   t_id       object
 1   parent_id  object
 2   season     object
 3   episode    object
dtypes: object(4)
memory usage: 177.0+ MB


**Just checking if numbers in season and episode are `str`or `int`**

In [86]:
for value in df_episode["season"]:
    # If the value is not a str and not null then print. If nothing gets printed, all vals are strings
    if type(value) != str and str(value) != "nan": 
        print(type(value))
    
# Just making sure the precedent comment is correct. Index choosen based on output of 4.4.2
print(type(df_episode.loc[0, "season"]))
print(type(df_episode.loc[0, "episode"]))

<class 'str'>
<class 'str'>


### Export Episodes 

Now that we have correct missing values and ids, let's export it!

In [87]:
df_episode.to_csv("data/episode.csv", index=False)
print("Episode table exported!")

# os.remove(directory + "/title_episode.tsv")

Episode table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Crew

### Import file

In [88]:
df_crew = pd.read_csv("data/TSVs/title_crew.tsv", delimiter="\t")
import_shape = df_crew.shape
import_shape

(7951502, 3)

In [89]:
df_crew.columns = ['t_id', 'directors', 'writers']

### Replacing missing values & id of id_columns


In [90]:
df_crew = clean_df(df_crew)
df_crew.head()

,t_id,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


### Flatten the table 

Some ids have multiple directors and/or writers, I want each row to have one value only. I'm basically redoing what I did in `4.1.5`

Need to find a way to fasten this

In [91]:
# Flattenning both columns into two dataframes
df_directors = df_crew[["t_id", "directors"]].set_index("t_id")["directors"].str.split(",")\
                                         .explode().to_frame().reset_index()

df_writers = df_crew[["t_id", "writers"]].set_index("t_id")["writers"].str.split(",")\
                                         .explode().to_frame().reset_index()


# Merging both dfs on the ID and as outer as we still want a director if there is no writer
df_crew = df_directors.dropna().merge(df_writers.dropna(), how="outer", on="t_id")

### Export Crew 

Now that we have correct missing values and ids, let's export it!

In [92]:
export_shape = df_crew.shape
print(f"Import shape: {import_shape} \nExport shape: {export_shape}")

Import shape: (7951502, 3) 
Export shape: (20483997, 3)


In [93]:
df_crew.to_csv("data/crew.csv", index=False)
print("Crew table exported!")

# os.remove(directory + "/title_crew.tsv")

Crew table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Akas

### Import file

In [94]:
df_akas = pd.read_csv("data/TSVs/title_akas.tsv", delimiter="\t")
df_akas.shape

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(26444606, 8)

In [95]:
df_akas.columns = ['t_id', 'ordering', 'title', 'region', 'language', 
                   'types', 'attributes', 'isoriginal']

### Replacing missing values & id of id_columns


In [96]:
df_akas = clean_df(df_akas)
df_akas.head()

,t_id,ordering,title,region,language,types,attributes,isoriginal
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


### Changing data types 

In [97]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26444606 entries, 0 to 26444605
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   t_id        object
 1   ordering    int64 
 2   title       object
 3   region      object
 4   language    object
 5   types       object
 6   attributes  object
 7   isoriginal  object
dtypes: int64(1), object(7)
memory usage: 1.6+ GB


#### Changing isoriginal from `object` to `int`

In [98]:
df_akas = series_to_int(df_akas, "isoriginal")

**Making sure `isoriginal` dtype is int**

In [99]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26444606 entries, 0 to 26444605
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   t_id        object
 1   ordering    int64 
 2   title       object
 3   region      object
 4   language    object
 5   types       object
 6   attributes  object
 7   isoriginal  Int64 
dtypes: Int64(1), int64(1), object(6)
memory usage: 1.6+ GB


In [100]:
df_akas["isoriginal"].value_counts()

0    25886759
1      555658
Name: isoriginal, dtype: Int64

### Export Akas 

Now that we have correct missing values and ids, let's export it!

In [101]:
df_akas.drop("ordering", axis=1).to_csv("data/akas.csv", index=False)
print("Akas table exported!")

# os.remove(directory + "/title_akas.tsv")

Akas table exported!


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Names

### Import file

In [102]:
df_names = pd.read_csv("data/TSVs/name_basics.tsv", delimiter="\t")
import_shape = df_names.shape

In [103]:
df_names.columns = ['n_id', 'name', 'birth', 'death', 'profession','known_for']

### Replacing missing values & id of id_columns


In [104]:
df_names = clean_df(df_names)
df_names.head()

,n_id,name,birth,death,profession,known_for
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0050419,tt0031983,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0071877,tt0117057"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0056404,tt0057345,tt0049189,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0078723,tt0077975,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0083922,tt0060827"


### Change data types

In [105]:
df_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10968938 entries, 0 to 10968937
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   n_id        object
 1   name        object
 2   birth       object
 3   death       object
 4   profession  object
 5   known_for   object
dtypes: object(6)
memory usage: 502.1+ MB


#### Changing birth and death from ` ` to `datetime`

In [106]:
df_names["birth"] = pd.to_datetime(df_names["birth"], format='%Y', errors='coerce')
df_names["death"] = pd.to_datetime(df_names["death"], format='%Y', errors='coerce')

In [107]:
df_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10968938 entries, 0 to 10968937
Data columns (total 6 columns):
 #   Column      Dtype         
---  ------      -----         
 0   n_id        object        
 1   name        object        
 2   birth       datetime64[ns]
 3   death       datetime64[ns]
 4   profession  object        
 5   known_for   object        
dtypes: datetime64[ns](2), object(4)
memory usage: 502.1+ MB


### Known_for flatten

In [108]:
df_known_for = df_names[["n_id", "known_for"]].set_index("n_id")["known_for"].str.split(",")\
                                         .explode().to_frame().reset_index().dropna()

df_known_for.shape

(17114065, 2)

### Export both tables: Names and Known_for

Now that we have correct missing values and ids, let's export it!

In [109]:
df_known_for.to_csv("data/known_for.csv", index=False)
print("Known_for table exported!")

# We don't need genre column in basics as we have a new df for it 
df_names.drop("known_for", axis=1).to_csv("data/names.csv", index=False)
print("Name table exported!")

# os.remove(directory + "/name_basics.tsv")

Known_for table exported!
Name table exported!


In [110]:
# This is just for me when I run the script fully and I want to know when it's done. From my bed. 
on()
diff = round((time.time() - start)/60)
print(f"It took {diff} minutes")
print("✅"*1000)

It took 17 minutes
✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅

In [30]:
df_basics.head()

NameError: name 'df_basics' is not defined

# Merge all tables 


In [ ]:
# set(df_known_for["t_id"]).intersection(df_episode["t_id"])

df_names[df_names["n_id"].isin(df_principals.loc[:20, "n_id"])]["n_id"]

In [ ]:
# List that contains the series to try out the merge.
movies_to_keep = df_names[df_names["n_id"].isin(df_principals.loc[:200000, "n_id"])]["n_id"]
size = []

df_basics_sample = df_basics[df_basics["t_id"].isin(movies_to_keep)]
size.append(["Basics", df_basics_sample.shape, "t_id"])

df_genres_sample = df_genres[df_genres["t_id"].isin(movies_to_keep)]
size.append(["Genres", df_genres_sample.shape, "t_id"])

df_principals_sample = df_principals[df_principals["t_id"].isin(movies_to_keep)]
size.append(["Principals", df_principals_sample.shape, "t_id"])

df_ratings_sample = df_ratings[df_ratings["t_id"].isin(movies_to_keep)]
size.append(["Ratings", df_ratings_sample.shape, "t_id"])

df_episode_sample = df_episode[df_episode["t_id"].isin(movies_to_keep)]
size.append(["Episode", df_episode_sample.shape, "t_id"])

df_crew_sample = df_crew[df_crew["t_id"].isin(movies_to_keep)]
size.append(["Crew", df_crew_sample.shape, "t_id"])

df_akas_sample = df_akas[df_akas["t_id"].isin(movies_to_keep)]
size.append(["Akas", df_akas_sample.shape, "t_id"])

df_known_for_sample = df_known_for[df_known_for["t_id"].isin(movies_to_keep)]
size.append(["Known_for", df_known_for_sample.shape, "t_id"])

df_names_sample = df_names[df_names["n_id"].isin(df_principals_sample.head()["n_id"])]
size.append(["Names", df_names_sample.shape, "n_id"])



pd.DataFrame(size, columns=["Dataset", "size", "merge_on"])

## Merging

In [ ]:
pd.set_option("display.max_columns", None)

start = time.time()

df_bg = df_basics.merge(df_genres, on="t_id", how="outer")
print(f"2 | {df_bg.shape[0]:,d} | {round(time.time() - start)}s")
df_bgp = df_bg.merge(df_principals, on="t_id", how="outer")
print(f"3 | {df_bgp.shape[0]:,d} | {round(time.time() - start)}s")
df_bgpr = df_bgp.merge(df_ratings, on="t_id", how="outer")
print(f"4 | {df_bgpr.shape[0]:,d} | {round(time.time() - start)}s")
df_bgpre = df_bgpr.merge(df_episode, on="t_id", how="outer")
print(f"5 | {df_bgpre.shape[0]:,d} | {round(time.time() - start)}s")
df_bgprec = df_bgpre.merge(df_crew, on="t_id", how="outer")
print(f"6 | {df_bgprec.shape[0]:,d} | {round(time.time() - start)}s")
df_bgpreca = df_bgprec.merge(df_akas, on="t_id", how="outer")
print(f"7 | {df_bgpreca.shape[0]:,d} | {round(time.time() - start)}s")
df_bgprecak = df_bgpreca.merge(df_known_for, on=["t_id", "n_id"], how="outer")
print(f"8 | {df_bgprecak.shape[0]:,d} | {round(time.time() - start)}s")
df_bgprecakn = df_bgpreca.merge(df_names, on="n_id", how="outer")
print(f"9 | {df_bgprecakn.shape[0]:,d} | {round(time.time() - start)}s")
        

print(f"✅"*1000)

In [ ]:
all_dfs = [df_basics, df_genres, df_principals, df_ratings, 
           df_episode, df_crew, df_akas, df_known_for, df_names]

all_dfs_sample = [df_basics_sample, df_genres_sample, df_principals_sample, df_ratings_sample, 
           df_episode_sample, df_crew_sample, df_akas_sample, df_known_for_sample, df_names_sample]



for x in all_dfs: 
    print(x.shape)

### Import dfs

In [ ]:
df_basics = pd.read_csv("data/basics.csv", low_memory=False)
df_genres = pd.read_csv("data/genres.csv", low_memory=False)
df_principals = pd.read_csv("data/principals.csv", low_memory=False)
df_ratings = pd.read_csv("data/ratings.csv", low_memory=False)
df_episode = pd.read_csv("data/episode.csv", low_memory=False)
df_crew = pd.read_csv("data/crew.csv", low_memory=False)
df_akas = pd.read_csv("data/akas.csv", low_memory=False)
df_known_for = pd.read_csv("data/known_for.csv", low_memory=False)
df_names = pd.read_csv("data/names.csv", low_memory=False)

In [ ]:
df_known_for_sample.head()

In [79]:
df_episode = pd.read_csv("data/crew.csv")

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Sample dfs based on year

In [4]:
def sample_year(start_year=2021, limit=None, end_year=2021, sample=True, export=True):
    # This function can also be used to load all dfs at once! 
    global df_basics
    global df_genres
    global df_principals 
    global df_ratings
    global df_episode
    global df_crew
    global df_akas
    global df_known_for
    global df_names
    global df_basics_sample
    global df_genres_sample
    global df_principals_sample 
    global df_ratings_sample
    global df_episode_sample
    global df_crew_sample
    global df_akas_sample
    global df_known_for_sample
    global df_names_sample
    global movies_id
    global directory
    
    # Create sub-folder that will contain the new data
    directory = f"data/{start_year} - {end_year}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # To measure how long it takes
    start = time.time()
    
    # Import the CSVs
    print("Importing")
    df_basics = pd.read_csv("data/basics.csv", parse_dates=["start_year", "end_year"])
    print("Importing: Basics", " "*20, end="\r")
    df_genres = pd.read_csv("data/genres.csv")
    print("Importing: Genres", " "*20, end="\r")
    df_principals = pd.read_csv("data/principals.csv")
    print("Importing: Principals", " "*20, end="\r")
    df_ratings = pd.read_csv("data/ratings.csv")
    print("Importing: Ratings", " "*20, end="\r")
    df_episode = pd.read_csv("data/episode.csv")
    print("Importing: Episodes", " "*20, end="\r")
    df_crew = pd.read_csv("data/crew.csv", dtype={"t_id": str, "directors": str, "writers": str})
    print("Importing: Crew", " "*20, end="\r")
    df_akas = pd.read_csv("data/akas.csv")
    print("Importing: Akas", " "*20, end="\r")
    df_known_for = pd.read_csv("data/known_for.csv")
    print("Importing: Known_for", " "*20, end="\r")
    df_names = pd.read_csv("data/names.csv")
    import_time = round(time.time()-start)
    print(f"Imported! {import_time}s")
    
    movies_id = df_basics[df_basics["start_year"].dt.year.isin(list(range(start_year, end_year + 1)))]["t_id"].to_list()
    if limit != None:
        movies_id = movies_id[:limit]
        
    if sample == True:
        df_basics_sample = df_basics[df_basics["t_id"].isin(movies_id)]
        df_genres_sample = df_genres[df_genres["t_id"].isin(movies_id)]
        df_principals_sample = df_principals[df_principals["t_id"].isin(movies_id)]
        df_ratings_sample = df_ratings[df_ratings["t_id"].isin(movies_id)]
        df_episode_sample = df_episode[df_episode["t_id"].isin(movies_id)]
        df_crew_sample = df_crew[df_crew["t_id"].isin(movies_id)]
        df_akas_sample = df_akas[df_akas["t_id"].isin(movies_id)]
        df_known_for_sample = df_known_for[df_known_for["n_id"].isin(df_principals_sample["n_id"])]
        df_names_sample = df_names[df_names["n_id"].isin(df_principals_sample["n_id"])]
        sample_time = round(time.time()-start) - import_time
        print(f"Sampled! {sample_time}s")
        
    
    if export == True:
        df_basics_sample.to_csv(directory + f"/basics_{start_year}-{end_year}.csv", index=False)
        print("Export: Basics", " "*20, end="\r")
        df_genres_sample.to_csv(directory + f"/genres_{start_year}-{end_year}.csv", index=False)
        print("Export: Genres", " "*20, end="\r")
        df_principals_sample.to_csv(directory + f"/principals_{start_year}-{end_year}.csv", index=False)
        print("Export: Principals", " "*20, end="\r")
        df_ratings_sample.to_csv(directory + f"/ratings_{start_year}-{end_year}.csv", index=False)
        print("Export: Ratings", " "*20, end="\r")
        df_episode_sample.to_csv(directory + f"/episode_{start_year}-{end_year}.csv", index=False)
        print("Export: Episode", " "*20, end="\r")
        df_crew_sample.to_csv(directory + f"/crew_{start_year}-{end_year}.csv", index=False)
        print("Export: Crew", " "*20, end="\r")
        df_akas_sample.to_csv(directory + f"/akas_{start_year}-{end_year}.csv", index=False)
        print("Export: Akas", " "*20, end="\r")
        df_known_for_sample.to_csv(directory + f"/known_for_{start_year}-{end_year}.csv", index=False)
        print("Export: Known_for", " "*20, end="\r")
        df_names_sample.to_csv(directory + f"/names_{start_year}-{end_year}.csv", index=False)
        export_time = round(time.time()-start) - sample_time
        print(f"Exported! {export_time}s", " "*20)
        
    print("Done!")



In [18]:
sample_year(2019, 2021)

Importing
Imported! 394swn_for                      
Movied!
Sampled! 124s
Exported! 478s                         


In [36]:
movies_id = df_basics[df_basics["start_year"].dt.year.isin(list(range(2019, 2020 + 1)))]["t_id"].to_list()

df_episode = pd.read_csv("data/episode.csv")

df_episode_sample = df_episode[df_episode["t_id"].isin(movies_id)]

In [40]:
start_year = 2019
end_year = 2020 + 1

df_episode_sample.to_csv(f"data/{start_year} - {end_year}" + f"/episode_{start_year}-{end_year}.csv", index=False)
df_episode_sample


In [41]:
df_episode_sample.head()

,t_id,parent_id,season,episode
57992,tt0547753,tt0138228,82.0,16.0
58003,tt0547764,tt0138228,82.0,17.0
58496,tt0548263,tt0138228,82.0,29.0
58566,tt0548336,tt0138228,82.0,4.0
58577,tt0548347,tt0138228,82.0,5.0


# Merge


In [ ]:
print("Importing")
df_basics = pd.read_csv("data/basics.csv", parse_dates=["start_year", "end_year"])
print("Importing: Basics", " "*20, end="\r")
df_genres = pd.read_csv("data/genres.csv")
print("Importing: Genres", " "*20, end="\r")
df_principals = pd.read_csv("data/principals.csv")
print("Importing: Principals", " "*20, end="\r")
df_ratings = pd.read_csv("data/ratings.csv")
print("Importing: Ratings", " "*20, end="\r")
df_episode = pd.read_csv("data/episode.csv")
print("Importing: Episodes", " "*20, end="\r")
df_crew = pd.read_csv("data/crew.csv", dtype={"t_id": str, "directors": str, "writers": str})
print("Importing: Crew", " "*20, end="\r")
df_akas = pd.read_csv("data/akas.csv")
print("Importing: Akas", " "*20, end="\r")
df_known_for = pd.read_csv("data/known_for.csv")
print("Importing: Known_for", " "*20, end="\r")
df_names = pd.read_csv("data/names.csv")
import_time = round(time.time()-start)
print(f"Imported! {import_time}s")

In [ ]:
df_basics_merge = df_basics.drop(["original_title", "for_adult"], axis=1)

df_principals_merge = df_principals.drop(["character_played", "job_title"], axis=1)

df_ratings_merge = df_ratings

df_episode_merge = df_episode

df_crew_merge = df_crew

df_names_merge = df_names.drop(["profession", "known_for"], axis=1)


In [7]:
pd.set_option("display.max_columns", None)
dfm1 = df_principals_merge.merge(df_basics_merge, on="t_id", how="outer")
print(dfm1.shape)
dfm1.head()

(45719706, 8)


,t_id,n_id,job_category,type,primary_title,start_year,end_year,runtime_mn
0,tt0000001,nm1588970,self,short,Carmencita,1894-01-01,NaT,1.0
1,tt0000001,nm0005690,director,short,Carmencita,1894-01-01,NaT,1.0
2,tt0000001,nm0374658,cinematographer,short,Carmencita,1894-01-01,NaT,1.0
3,tt0000002,nm0721526,director,short,Le clown et ses chiens,1892-01-01,NaT,5.0
4,tt0000002,nm1335271,composer,short,Le clown et ses chiens,1892-01-01,NaT,5.0


In [8]:
dfm2 = dfm1.merge(df_ratings_merge, on="t_id", how="outer")
print(dfm2.shape)
dfm2.head()

(45719706, 10)


,t_id,n_id,job_category,type,primary_title,start_year,end_year,runtime_mn,rating,votes
0,tt0000001,nm1588970,self,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0
1,tt0000001,nm0005690,director,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0
2,tt0000001,nm0374658,cinematographer,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0
3,tt0000002,nm0721526,director,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0
4,tt0000002,nm1335271,composer,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0


In [9]:
dfm3 = dfm2.merge(df_episode_merge, on="t_id", how="outer")
print(dfm3.shape)
dfm3.head()

(45719706, 13)


,t_id,n_id,job_category,type,primary_title,start_year,end_year,runtime_mn,rating,votes,parent_id,season,episode
0,tt0000001,nm1588970,self,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN
1,tt0000001,nm0005690,director,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN
2,tt0000001,nm0374658,cinematographer,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN
3,tt0000002,nm0721526,director,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0,NaN,NaN,NaN
4,tt0000002,nm1335271,composer,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0,NaN,NaN,NaN


In [ ]:
dfm5 = dfm4.merge(df_ratings_merge, on="t_id", how="outer")
print(dfm5.shape)
dfm5.head()

In [10]:
dfm4 = dfm3.merge(df_crew_merge, on="t_id", how="outer")
print(dfm4.shape)
dfm4.head()

(192998591, 15)


,t_id,n_id,job_category,type,primary_title,start_year,end_year,runtime_mn,rating,votes,parent_id,season,episode,directors,writers
0,tt0000001,nm1588970,self,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN,nm0005690,NaN
1,tt0000001,nm0005690,director,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN,nm0005690,NaN
2,tt0000001,nm0374658,cinematographer,short,Carmencita,1894-01-01,NaT,1.0,5.7,1707.0,NaN,NaN,NaN,nm0005690,NaN
3,tt0000002,nm0721526,director,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0,NaN,NaN,NaN,nm0721526,NaN
4,tt0000002,nm1335271,composer,short,Le clown et ses chiens,1892-01-01,NaT,5.0,6.1,210.0,NaN,NaN,NaN,nm0721526,NaN


In [ ]:
dfm6 = dfm5.merge(df_names_merge, on="n_id", how="outer")
print(dfm6.shape)
dfm6.head()